In [2]:
!pip install sentence-transformers

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.9/275.9 kB 3.1 MB/s eta 0:00:00a 0:00:01


In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https:/

In [2]:
data = pd.read_csv('cleaned_data.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102863 entries, 0 to 102862
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   joke_type  102863 non-null  int64 
 1   score      102863 non-null  int64 
 2   title      102862 non-null  object
 3   selftext   102850 non-null  object
dtypes: int64(2), object(2)
memory usage: 3.1+ MB


In [4]:
data['joke'] = data['title'] + ' ' + data['selftext']

In [5]:
data['joke_type'] = data['joke_type'].replace({3: 0, 2: 1})

In [6]:
df = data[['joke', 'joke_type']]

In [7]:
x = df[df['joke'].isnull()].index
df = df.drop(x)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 102849 entries, 0 to 102862
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   joke       102849 non-null  object
 1   joke_type  102849 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [22]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['joke'], df['joke_type'], test_size=0.2, stratify=df['joke_type'], random_state=42)

In [10]:
model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight, fast model

In [11]:
train_embeddings = model.encode(train_texts.tolist())
test_embeddings = model.encode(test_texts.tolist())

print("Train Embeddings Shape:", train_embeddings.shape)  # (num_samples, embedding_dim)

Train Embeddings Shape: (82279, 384)


In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Train Logistic Regression on embeddings
clf_lr = LogisticRegression(class_weight='balanced')
clf_lr.fit(train_embeddings, train_labels)

# Predict categories
pred_labels = clf_lr.predict(test_embeddings)

# Evaluate
print(classification_report(test_labels, pred_labels))

              precision    recall  f1-score   support

           0       0.17      0.47      0.25      3632
           1       0.82      0.52      0.64     16938

    accuracy                           0.51     20570
   macro avg       0.50      0.50      0.45     20570
weighted avg       0.71      0.51      0.57     20570



In [13]:
jester = pd.read_csv('jester_items.csv')

In [14]:
jokes_embed = model.encode(jester['jokeText'])

In [24]:
clf_lr.predict(jokes_embed)

array([0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1])

In [16]:
# Problem becasue of imbalanced dataset

In [18]:
tweets = pd.read_csv('funnytweeter.csv')

In [19]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44758 entries, 0 to 44757
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      44758 non-null  int64 
 1   Joke    44758 non-null  object
dtypes: int64(1), object(1)
memory usage: 699.5+ KB


In [20]:
# This is generally clean

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

# Train Decision Tree on embeddings
clf_dt = DecisionTreeClassifier(class_weight='balanced', random_state=42)
clf_dt.fit(train_embeddings, train_labels)

# Predict categories
pred_labels = clf_dt.predict(test_embeddings)

# Evaluate
print(classification_report(test_labels, pred_labels))
